# 🐺 Convert Ckpt to Safetensors

| | |
|:--|:-:|
| **SD WebUI** | [![Open in Colab](https://raw.githubusercontent.com/mesmerwolfies/MW-StableDiffusion/main/colab-badge.svg)](https://colab.research.google.com/github/mesmerwolfies/MW-StableDiffusion/blob/main/SDVN_WebUI_v3.ipynb)|
| **Download Tools** | [![Open in Colab](https://raw.githubusercontent.com/mesmerwolfies/MW-StableDiffusion/main/colab-badge.svg)](https://colab.research.google.com/github/mesmerwolfies/MW-StableDiffusion/blob/main/SDVN_Download_tools.ipynb)|



In [ ]:
#@title # ⚙️ 1. Settings

from google.colab import drive
drive.mount('/content/drive')

!pip install torch safetensors -q

import os
import torch
from safetensors.torch import save_file
import hashlib

# --- Hằng số ---
CKPT_STR = "ckpt"
SAFETENSORS_STR = "safetensors"
HASH_START = 0x100000
HASH_LENGTH = 0x10000

# --- Hàm Băm Tệp ---
def get_file_hash(filename):
    """Tính toán và trả về một phần hash SHA256 của tệp."""
    try:
        with open(filename, "rb") as file:
            m = hashlib.sha256()
            file.seek(HASH_START)
            m.update(file.read(HASH_LENGTH))
            return m.hexdigest()[0:8]
    except FileNotFoundError:
        print(f"CẢNH BÁO: Không tìm thấy tệp để băm: {filename}")
        return "N/A"
    except Exception as e:
        print(f"CẢNH BÁO: Không thể băm tệp {filename}: {e}")
        return "Lỗi"

# --- Hàm Tải Trọng số (cho .ckpt, luôn tải lên CPU) ---
def load_weights_from_ckpt(checkpoint_path):
    """Tải trọng số từ tệp .ckpt. Luôn map về CPU."""
    try:
        with torch.no_grad():
            weights = torch.load(checkpoint_path, map_location=torch.device('cpu'))
            if "state_dict" in weights:
                weights = weights["state_dict"]
                if "state_dict" in weights:
                    weights.pop("state_dict")
            return weights
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy tệp - {checkpoint_path}")
    except (RuntimeError, EOFError):
        print(f"LỖI: KHÔNG THỂ TẢI CHECKPOINT: Tệp bị hỏng - {checkpoint_path}")
    except (AttributeError, KeyError):
        print(f"LỖI: TỆP KHÔNG ĐƯỢC HỖ TRỢ - {checkpoint_path}")
    except Exception as e:
        print(f'Lỗi khi tải trọng số từ {checkpoint_path}: {e}')
    return None

# --- Hàm Chuyển đổi Tệp Đơn ---
def convert_single_ckpt_to_st_on_drive(input_ckpt_drive_path, output_safetensors_custom_name=None):
    """
    Chuyển đổi một tệp .ckpt từ Google Drive sang .safetensors và lưu lại vào cùng thư mục.
    Args:
        input_ckpt_drive_path (str): Đường dẫn đầy đủ đến tệp .ckpt đầu vào trên Google Drive.
        output_safetensors_custom_name (str, optional):
            - Nếu được cung cấp (ví dụ: "my_model.safetensors" hoặc "my_model"),
              tên này sẽ được sử dụng cho tệp đầu ra (sẽ tự động thêm .safetensors nếu thiếu).
            - Nếu None hoặc trống, tên tệp đầu ra sẽ giống tên tệp đầu vào nhưng có phần mở rộng .safetensors.
    """
    print(f"Bắt đầu chuyển đổi tệp đơn: {input_ckpt_drive_path}")
    if not os.path.exists(input_ckpt_drive_path):
        print(f"LỖI: Không tìm thấy tệp đầu vào: {input_ckpt_drive_path}")
        return False
    if not input_ckpt_drive_path.lower().endswith(f".{CKPT_STR}"):
        print(f"LỖI: Tệp đầu vào phải có phần mở rộng .{CKPT_STR}: {input_ckpt_drive_path}")
        return False

    original_hash = get_file_hash(input_ckpt_drive_path)
    print(f"Đang tải trọng số từ {os.path.basename(input_ckpt_drive_path)} [{original_hash}] (lên CPU)..." )
    weights = load_weights_from_ckpt(input_ckpt_drive_path)
    if weights is None:
        print(f"Không thể tải trọng số cho {input_ckpt_drive_path}. Bỏ qua.")
        return False

    input_dir = os.path.dirname(input_ckpt_drive_path)
    input_filename_no_ext = os.path.splitext(os.path.basename(input_ckpt_drive_path))[0]
    final_output_filename_no_ext = ""

    if output_safetensors_custom_name and output_safetensors_custom_name.strip():
        custom_name_stripped = output_safetensors_custom_name.strip()
        # Nếu người dùng nhập cả đuôi .safetensors, loại bỏ nó để chuẩn hóa
        if custom_name_stripped.lower().endswith(f".{SAFETENSORS_STR}"):
            final_output_filename_no_ext = os.path.splitext(custom_name_stripped)[0]
        else:
            final_output_filename_no_ext = custom_name_stripped
    else:
        final_output_filename_no_ext = input_filename_no_ext

    final_output_path = os.path.join(input_dir, f"{final_output_filename_no_ext}.{SAFETENSORS_STR}")

    print(f"Đang cố gắng lưu tệp đã chuyển đổi vào: {final_output_path}")
    try:
        save_file(weights, final_output_path)
        new_hash = get_file_hash(final_output_path)
        print(f"Đã chuyển đổi thành công {os.path.basename(input_ckpt_drive_path)} [{original_hash}] sang {SAFETENSORS_STR}.")
        print(f"Đã lưu thành {final_output_path} [{new_hash}].")
        return True
    except Exception as e:
        print(f"LỖI: Không thể lưu tệp .safetensors vào {final_output_path}: {e}")
        return False

# --- Hàm Chuyển đổi Thư mục ---
def convert_folder_ckpt_to_st_on_drive(input_folder_drive_path):
    """
    Chuyển đổi tất cả các tệp .ckpt trong một thư mục (và các thư mục con của nó) trên Google Drive sang .safetensors.
    Các tệp đã chuyển đổi được lưu trong cùng thư mục với tệp .ckpt gốc tương ứng.
    """
    print(f"--- Bắt đầu chuyển đổi thư mục: {input_folder_drive_path} ---")
    if not os.path.isdir(input_folder_drive_path):
        print(f"LỖI: Thư mục đầu vào không tồn tại hoặc không phải là thư mục: {input_folder_drive_path}")
        return

    ckpt_files_found = []
    for root, _, files in os.walk(input_folder_drive_path):
        for file in files:
            if file.lower().endswith(f".{CKPT_STR}"):
                ckpt_files_found.append(os.path.join(root, file))

    if not ckpt_files_found:
        print(f"Không tìm thấy tệp .{CKPT_STR} nào trong thư mục '{input_folder_drive_path}' (và các thư mục con) để chuyển đổi.")
        print("--- Kết thúc xử lý thư mục (không có tệp) ---")
        return

    print(f"Tìm thấy {len(ckpt_files_found)} tệp .{CKPT_STR} để chuyển đổi.")
    overall_success_count = 0
    overall_failure_count = 0

    for i, ckpt_file_path in enumerate(ckpt_files_found):
        print(f"\nĐang xử lý tệp {i+1}/{len(ckpt_files_found)}: {ckpt_file_path}")
        # Đối với chuyển đổi thư mục, tên tệp đầu ra tùy chỉnh không áp dụng từng tệp,
        # chỉ có tùy chọn thêm hậu tố.
        # Tên tệp đầu ra sẽ dựa trên tên gốc, lưu cùng vị trí.
        input_dir_of_current_file = os.path.dirname(ckpt_file_path)
        output_filename_base = os.path.splitext(os.path.basename(ckpt_file_path))[0]

        # Hàm convert_single_ckpt_to_st_on_drive sẽ tự xử lý việc thêm .safetensors
        # và lưu vào đúng thư mục (input_dir_of_current_file)
        # Chúng ta truyền output_filename_base làm tên tùy chỉnh.
        success = convert_single_ckpt_to_st_on_drive(ckpt_file_path, output_safetensors_custom_name=output_filename_base)
        if success:
            overall_success_count += 1
        else:
            overall_failure_count += 1

    print(f"\n--- Xử lý thư mục hoàn tất ---")
    print(f"Tổng cộng thành công: {overall_success_count} tệp. Thất bại: {overall_failure_count} tệp.")


In [ ]:
#@title # 🚀 2. Run

#@markdown ### 📁 Chuyển đổi thư mục

#@markdown Tất cả các tệp `.ckpt` trong thư mục này và các thư mục con của nó sẽ được chuyển đổi.

folder_on_drive = "/content/drive/MyDrive/MesmerWolfies/MW-StableDiffusion/Model/ckpt" #@param {type:"string"}
convert_folder = False #@param {type:"boolean"}

#@markdown ### 💽 Chuyển đổi tệp đơn lẻ

ckpt_file_on_drive = "/content/drive/MyDrive/MesmerWolfies/MW-StableDiffusion/Model/ckpt/" #@param {type:"string"}

#@markdown Chỉ tên tệp. Nếu để trống, tên sẽ giống tệp đầu vào
output_safetensors_name = "" #@param {type:"string"}

input_folder_stripped = folder_on_drive.strip() if convert_folder else ""
input_file_stripped = _ckpt_file_on_drive.strip()

if input_folder_stripped: # Ưu tiên chuyển đổi thư mục
    print("Ưu tiên: Chuyển đổi thư mục")
    if not os.path.isdir(input_folder_stripped):
        print(f"LỖI (Thư mục): Đường dẫn đầu vào không phải là thư mục hoặc không tồn tại: {input_folder_stripped}")
    else:
        convert_folder_ckpt_to_st_on_drive(input_folder_stripped)
elif input_file_stripped: # Nếu không có thư mục, thử chuyển đổi tệp đơn
    print("Chế độ: Chuyển đổi tệp đơn lẻ")
    if not os.path.exists(input_file_stripped):
        print(f"LỖI (Tệp đơn): Không tìm thấy tệp .ckpt đầu vào tại: {input_file_stripped}")
    elif not input_file_stripped.lower().endswith(".ckpt"):
        print(f"LỖI (Tệp đơn): Tệp đầu vào phải là tệp .ckpt. Đường dẫn: {input_file_stripped}")
    else:
        output_name = output_safetensors_name.strip() if output_safetensors_name.strip() else None
        convert_single_ckpt_to_st_on_drive(input_file_stripped, output_name)
else:
    print("LỖI: Vui lòng cung cấp đường dẫn tệp .ckpt HOẶC đường dẫn thư mục để chuyển đổi.")
